In [3]:
import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr
from IPython.display import display
from pystac_client import Client
import geopandas as gpd
import requests
from pystac.extensions.eo import EOExtension as eo
from odc.stac import configure_rio, stac_load

import pickle
from contextlib import contextmanager  
import rasterio
from rasterio import Affine, MemoryFile
from rasterio.enums import Resampling
from rasterio.windows import Window
from rasterio.features import geometry_mask, geometry_window
from rasterio.plot import show, reshape_as_raster, reshape_as_image

In [4]:
stac_list_path = '/media/laserglaciers/upernavik/iceberg_py/sam/stac_nc/kanger_netcdf.nc'
# with open(stac_list_path, 'rb') as src:
#     stac_list = pickle.load(src)
# grid_path = '/media/laserglaciers/upernavik/iceberg_py/geoms/helheim/melange_box_grid_utm24.shp'
# grid = gpd.read_file(grid_path)

image_stac = xr.open_dataset(stac_list_path)


In [5]:
image_stac

<xarray.Dataset> Size: 332MB
Dimensions:      (y: 3399, x: 4071, time: 4)
Coordinates:
  * y            (y) float64 27kB 7.618e+06 7.618e+06 ... 7.584e+06 7.584e+06
  * x            (x) float64 33kB 4.944e+05 4.944e+05 ... 5.351e+05 5.351e+05
  * time         (time) datetime64[ns] 32B 2016-07-13T14:19:52.026000 ... 202...
Data variables:
    spatial_ref  int32 4B ...
    red          (time, y, x) uint16 111MB ...
    green        (time, y, x) uint16 111MB ...
    blue         (time, y, x) uint16 111MB ...

In [6]:
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)
display(client)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33411,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.45 GiB
Comm: tcp://127.0.0.1:44979,Total threads: 3
Dashboard: http://127.0.0.1:39739/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:37849,


In [7]:
resolution = 10
SHRINK = 1
if client.cluster.workers[0].memory_manager.memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # running on Binder with 2Gb RAM

if SHRINK > 1:
    resolution = resolution * SHRINK

# image_stac = stac_load(
#     stac_list,
#     bands=["red", "green", "blue"],
#     resolution=resolution,
#     chunks={},
#     patch_url=pc.sign,
#     # force dtype and nodata
#     dtype="uint16",
#     nodata=0,
#     geopolygon=grid
# )

In [8]:
image_stac.red.shape

(4, 3399, 4071)

In [9]:
#https://rasterio.groups.io/g/main/topic/memoryfile_workflow_should/32634761
@contextmanager
def mem_raster(data, **profile):
    with MemoryFile() as memfile:
        with memfile.open(**profile) as dataset_writer:
            dataset_writer.write(data)
 
        with memfile.open() as dataset_reader:
            yield dataset_reader


In [47]:
affine = image_stac.odc.affine
profile = {'driver':'GTiff', 'count':3, 
                      'transform':affine, 'crs':image_stac.odc.crs, 
                      'width':image_stac.red.shape[2], 'height': image_stac.red.shape[1], 
                      'dtype':np.float64
            }
rbg_path = '/media/laserglaciers/upernavik/iceberg_py/infiles/kanger/rbg/'


In [48]:
for time in image_stac.time:
    rgb = np.dstack((image_stac.red.sel(time=time).values,
                     image_stac.green.sel(time=time).values,
                     image_stac.blue.sel(time=time).values))
    rgb_raster = reshape_as_raster(rgb)
    date = str(image_stac.red.sel(time=time).time.dt.date.data)
    with mem_raster(rgb_raster, **profile) as ds:
        ds_read = ds.read([1,2,3])

        with rasterio.open(f'{rbg_path}{date}.tif', mode='w', **profile) as dst:
            dst.write(ds_read)

        # show(ds.read([1,2,3]),adjust=True)

    

In [24]:
image_stac.time[:1]

<xarray.DataArray 'time' (time: 1)> Size: 8B
array(['2016-04-04T14:19:52.030000000'], dtype='datetime64[ns]')
Coordinates:
    spatial_ref  int32 4B 32624
  * time         (time) datetime64[ns] 8B 2016-04-04T14:19:52.030000